In [1]:
import numpy as np
from scipy import linalg as LA

In [2]:
M = np.genfromtxt('M.csv', delimiter=',')
size = M.shape[0]

print("M")
print(M)
print("size")
print(size)

M
[[0.  0.  0.6 0.  0.  0.  0.1 0.  0.  0. ]
 [0.  0.  0.  0.  0.3 0.  0.  0.  0.  0. ]
 [0.  0.2 0.  0.  0.3 0.1 0.  0.  0.1 0. ]
 [0.  0.  0.  0.  0.  0.1 0.  0.5 0.4 0. ]
 [0.  0.  0.  0.  0.  0.4 0.  0.  0.  0. ]
 [0.6 0.  0.  0.  0.  0.  0.5 0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.4 0.  0.1 0.5]
 [0.4 0.5 0.  0.2 0.3 0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.4 0.  0.  0.  0.5 0.  0. ]
 [0.  0.3 0.4 0.4 0.1 0.4 0.  0.  0.4 0.5]]
size
10


## Eigen Analysis

In [3]:
def eigen_analysis():    
    result_pairs = np.linalg.eig(M)
    eigen_values = result_pairs[0]
    eigen_vectors = result_pairs[1]    
    v1 = eigen_vectors[0]
    q_ = v1 / np.linalg.norm(v1)
    return np.array(q_)
    
p_from_eigen = eigen_analysis()
print(p_from_eigen)

[-0.12035   +0.j         -0.0974722 -0.0964816j  -0.0974722 +0.0964816j
 -0.05636152+0.31121603j -0.05636152-0.31121603j  0.33275266+0.20436279j
  0.33275266-0.20436279j -0.3779016 +0.j         -0.38715739+0.01149969j
 -0.38715739-0.01149969j]


## State Propagation

In [4]:
def state_propagation(q_0, t):
    q_i = q_0
    for i in range(t):
        q_i = np.matmul(M, q_i)
    return q_i
        
q_0 = np.zeros(size)
q_0[0] = 1
q_0 = q_0.T
t = 2048
p_from_stateprop = state_propagation(q_0, t)
print(p_from_stateprop)

[0.04832012 0.0175804  0.04136312 0.06662768 0.05860132 0.14650329
 0.23502244 0.05902418 0.05616316 0.2707943 ]


## Matrix Power

In [5]:
def matrix_power(q_0, t):
    q_ = np.matmul(np.linalg.matrix_power(M, t), q_0)
    return q_

q_0 = np.zeros(size)
q_0[0] = 1
q_0 = q_0.T
t = 2048
p_from_matrixpower = state_propagation(q_0, t)
print(p_from_matrixpower)

[0.04832012 0.0175804  0.04136312 0.06662768 0.05860132 0.14650329
 0.23502244 0.05902418 0.05616316 0.2707943 ]


## Random Walk

In [6]:
def random_walk(q_0, t0, t):
    # burn in period
    M_t = M.T
    idx_current_in = 0
    for i in range(t0):
        idx_current_in = np.random.choice(size, 1, p=M_t[idx_current_in])[0]
    
    s = np.zeros(size)
    for i in range(t):
        idx_current_in = np.random.choice(size, 1, p=M_t[idx_current_in])[0]
        s[idx_current_in] += 1
    q_ = np.divide(s, t)
    return q_


q_0 = np.zeros(size)
q_0[0] = 1
q_0 = q_0.T
t = 2048
t0 = 100
p_from_randomwalk = random_walk(q_0, t, t0)
print(p_from_randomwalk)

[0.02 0.02 0.03 0.08 0.06 0.15 0.25 0.09 0.03 0.27]


### 1A: Run each method and report the answers

In [7]:
print("Eigen Analysis")
print(p_from_eigen)
print("norm", np.linalg.norm(p_from_eigen))
print("sum", np.sum(p_from_eigen))
print()

print("State Propagation")
print(p_from_stateprop)
print("norm", np.linalg.norm(p_from_stateprop))
print("sum", np.sum(p_from_stateprop))
print()

print("Matrix Power")
print(p_from_matrixpower)
print("norm", np.linalg.norm(p_from_matrixpower))
print("sum", np.sum(p_from_matrixpower))
print()

print("Random Walk")
print(p_from_randomwalk)
print("norm", np.linalg.norm(p_from_randomwalk))
print("sum", np.sum(p_from_randomwalk))
print()

Eigen Analysis
[-0.12035   +0.j         -0.0974722 -0.0964816j  -0.0974722 +0.0964816j
 -0.05636152+0.31121603j -0.05636152-0.31121603j  0.33275266+0.20436279j
  0.33275266-0.20436279j -0.3779016 +0.j         -0.38715739+0.01149969j
 -0.38715739-0.01149969j]
norm 1.0
sum (-0.9147284788397201+0j)

State Propagation
[0.04832012 0.0175804  0.04136312 0.06662768 0.05860132 0.14650329
 0.23502244 0.05902418 0.05616316 0.2707943 ]
norm 0.41096793674714016
sum 1.0000000000000016

Matrix Power
[0.04832012 0.0175804  0.04136312 0.06662768 0.05860132 0.14650329
 0.23502244 0.05902418 0.05616316 0.2707943 ]
norm 0.41096793674714016
sum 1.0000000000000016

Random Walk
[0.02 0.02 0.03 0.08 0.06 0.15 0.25 0.09 0.03 0.27]
norm 0.4226109321823088
sum 1.0



### 1B: Rerun the Matrix Power and State Propagation techniques with q0 = [0.1, 0.1, . . . , 0.1]T. For what value of t is required to get as close to the true answer as the older initial state?

#### State Propagation

In [25]:
q_0 = np.empty(size).T
q_0.fill(0.1)

t = 2048
converge_p_from_stateprop_distri = state_propagation(q_0, t)

print("**************State Propagation: **************")
print("converged case when t == ", t)
print(converge_p_from_stateprop_distri)
print("norm: ", np.linalg.norm(converge_p_from_stateprop_distri))
print()

print("iterate from t = 1 to 100")

for t in range(1, 100):
    p_from_stateprop_distri = state_propagation(q_0, t)
    # print("case when t == ", t)
    # print(p_from_stateprop_distri)
    print("case when t == ", t, ", delta from converged result: ", np.linalg.norm(converge_p_from_stateprop_distri-p_from_stateprop_distri))
    # print()
print()
print()

**************State Propagation: **************
converged case when t ==  2048
[0.04832012 0.0175804  0.04136312 0.06662768 0.05860132 0.14650329
 0.23502244 0.05902418 0.05616316 0.2707943 ]
sum:  1.000000000000001
norm:  0.41096793674713994

iterate from t = 1 to 100
case when t ==  1 , delta from converged result:  0.17493434590763998
case when t ==  2 , delta from converged result:  0.11351543995478222
case when t ==  3 , delta from converged result:  0.05867270887367954
case when t ==  4 , delta from converged result:  0.031501864835394536
case when t ==  5 , delta from converged result:  0.017631111365310943
case when t ==  6 , delta from converged result:  0.011501514966775376
case when t ==  7 , delta from converged result:  0.007024912382931624
case when t ==  8 , delta from converged result:  0.003792159405752857
case when t ==  9 , delta from converged result:  0.0019485119459941543
case when t ==  10 , delta from converged result:  0.0011257986797107047
case when t ==  11 ,

#### Matrix Power

In [27]:
q_0 = np.empty(size).T
q_0.fill(0.1)

t = 2048
converge_p_from_matpow_distri = matrix_power(q_0, t)

print("**************Matrix Power: **************")
print("converged case when t == ", t)
print(converge_p_from_matpow_distri)
print("norm: ", np.linalg.norm(converge_p_from_matpow_distri))
print()

print("iterate from t = 1 to 100")

for t in range(1, 100):
    p_from_matpow_distri = matrix_power(q_0, t)
    # print("case when t == ", t)
    # print(p_from_matpow_distri)
    print("case when t == ", t, ", delta from converged result: ", np.linalg.norm(converge_p_from_matpow_distri-p_from_matpow_distri))
    # print()
print()
print()

**************Matrix Power: **************
converged case when t ==  2048
[0.04832012 0.0175804  0.04136312 0.06662768 0.05860132 0.14650329
 0.23502244 0.05902418 0.05616316 0.2707943 ]
norm:  0.41096793674716386

iterate from t = 1 to 100
case when t ==  1 , delta from converged result:  0.17493434590765075
case when t ==  2 , delta from converged result:  0.11351543995478824
case when t ==  3 , delta from converged result:  0.05867270887367867
case when t ==  4 , delta from converged result:  0.031501864835386126
case when t ==  5 , delta from converged result:  0.017631111365297596
case when t ==  6 , delta from converged result:  0.01150151496676444
case when t ==  7 , delta from converged result:  0.007024912382926099
case when t ==  8 , delta from converged result:  0.0037921594057542885
case when t ==  9 , delta from converged result:  0.0019485119460033926
case when t ==  10 , delta from converged result:  0.0011257986797240814
case when t ==  11 , delta from converged result: